In [ ]:
import sys
print(sys.path)

In [ ]:
pip install Cython

In [ ]:
pip install pclpy

In [1]:
import open3d as o3d
import cv2
import numpy as np

# Load the geometry file (PLY) with color information
point_cloud = o3d.io.read_point_cloud("C:/Users/islam/Desktop/+++neu+++/Boroskopbilder/SchaufelX_V3_Farbnetz.ply")

In [2]:
# Visualize the geometry
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
# Define the array of voxel coordinates
voxels = np.array([[-11.66883755, -21.77524567, 215.68687439],
  [-11.66729259, -21.79191971, 215.71546936]])

# Calculate the distance between the first two elements (points)
distance_1_2 = np.linalg.norm(voxels[0] - voxels[1])

print("Distance between first two elements (points):", distance_1_2)

In [4]:
# Convert the point cloud to a numpy array
points = np.asarray(point_cloud.points)
print(points)
colors = np.asarray(point_cloud.colors)
print(colors)
# Convert normalized color values to the 8-bit scale
colors_8bit = (colors * 255).astype(np.uint8)
print(colors_8bit)

[[-11.66883755 -21.77524567 215.68687439]
 [-11.66729259 -21.79191971 215.71546936]
 [-11.66682243 -21.72731972 215.68818665]
 ...
 [ -6.64744711  -9.505229   253.05174255]
 [ -6.78415251  -9.26166248 252.58708191]
 [ -6.78010273  -9.30448437 252.41690063]]
[[0.40784314 0.40784314 0.40784314]
 [0.41568627 0.41568627 0.41568627]
 [0.40784314 0.40784314 0.40784314]
 ...
 [0.63529412 0.63529412 0.63529412]
 [0.72156863 0.72156863 0.72156863]
 [0.63137255 0.63137255 0.63137255]]
[[104 104 104]
 [106 106 106]
 [104 104 104]
 ...
 [162 162 162]
 [184 184 184]
 [161 161 161]]


In [5]:
# You may need to adjust this threshold based on your data
white_threshold = 175

# Find indices of points with bright white markings
white_markings_indices = np.where(np.all(colors_8bit >= white_threshold, axis=1))[0]

print(len(white_markings_indices))
# Extract points with bright white markings
points_with_white_markings = points[white_markings_indices]
print(points_with_white_markings)

32644
[[-11.55315876 -22.01726723 217.29718018]
 [-11.55278873 -21.95064926 212.87155151]
 [-11.54926872 -21.93388557 213.05404663]
 ...
 [ -6.64232826  -9.59300423 252.60229492]
 [ -6.54929638  -9.76791191 252.92773438]
 [ -6.78415251  -9.26166248 252.58708191]]


In [6]:
# Assign a distinct color to points with white markings (e.g., red)
color_white_markings = [0,0,255]  # Red color
colors_with_white_markings = np.full((len(points_with_white_markings), 3), color_white_markings)

# Convert the array of new points and colors to an Open3D point cloud
new_point_cloud = o3d.geometry.PointCloud()
new_point_cloud.points = o3d.utility.Vector3dVector(points_with_white_markings)
new_point_cloud.colors = o3d.utility.Vector3dVector(colors_with_white_markings)

# # Visualize the original point cloud and the new points together
o3d.visualization.draw_geometries([new_point_cloud])
# # Convert the array of new points to an Open3D point cloud
# new_point_cloud = o3d.geometry.PointCloud()
# new_point_cloud.points = o3d.utility.Vector3dVector(points_with_white_markings)

# # Visualize the original point cloud and the new points together
# o3d.visualization.draw_geometries([point_cloud, new_point_cloud])

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

# Assuming 'points_with_white_markings' is your array of points

# Initialize the DBSCAN clustering algorithm
dbscan = DBSCAN(eps=0.12, min_samples=5)  # You may need to adjust 'eps' and 'min_samples'

# Fit the DBSCAN algorithm to the points
labels = dbscan.fit_predict(points_with_white_markings)

# Get the unique cluster labels
unique_labels = np.unique(labels)

In [ ]:
# Initialize a list to store the centroids of each cluster
centroids = []

# Iterate over each unique cluster label
for label in unique_labels:
    # Find the points belonging to the current cluster
    cluster_points = points_with_white_markings[labels == label]
    
    # Calculate the centroid of the cluster
    centroid = np.mean(cluster_points, axis=0)
    
    # Append the centroid to the list
    centroids.append(centroid)

# Convert the list of centroids to a numpy array
centroids = np.array(centroids)

# Convert values to normal floating-point notation
values_normal = np.array([[f'{val:.6f}' for val in row] for row in centroids])
print(values_normal)

In [ ]:
pip install pyquaternion

In [ ]:
# Define the text_3d function
def text_3d(text, pos, direction=None, degree=0.0, font='C:/Users/islam/Downloads/DejaVu Sans Mono for Powerline/DejaVu Sans Mono for Powerline.ttf', font_size=16):
    """
    Generate a 3D text point cloud used for visualization.
    :param text: content of the text
    :param pos: 3D xyz position of the text upper left corner
    :param direction: 3D normalized direction of where the text faces
    :param degree: in plane rotation of text
    :param font: Name of the font - change it according to your system
    :param font_size: size of the font
    :return: o3d.geoemtry.PointCloud object
    """
    if direction is None:
        direction = ( 1., 0., 0.)

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion

    font_obj = ImageFont.truetype(font, font_size)
    font_dim = font_obj.getsize(text)

    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 0), text, font=font_obj, fill=(0, 0, 0))
    img = np.asarray(img)
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    pcd = o3d.geometry.PointCloud()
    pcd.colors = o3d.utility.Vector3dVector(img[img_mask, :].astype(float) / 255.0)
    pcd.points = o3d.utility.Vector3dVector(indices / 100.0)

    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)
    return pcd

In [7]:
# Convert the array of centroids to an Open3D point cloud
centroids_point_cloud = o3d.geometry.PointCloud()
centroids_point_cloud.points = o3d.utility.Vector3dVector(centroids)

# Define the voxel size for downsampling
voxel_size = 0.2 #Ajust this value according to your downsampling requirements

# Downsample the original point cloud
downsampled_point_cloud = point_cloud.voxel_down_sample(voxel_size)

# Downsample the centroid point cloud
downsampled_centroids_point_cloud = centroids_point_cloud.voxel_down_sample(voxel_size)


# Create Visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window()

# Initialize lists for colors
centroid_colors = []

# Generate random color for each centroid
for _ in range(len(downsampled_centroids_point_cloud.points)):
    color = np.random.rand(3)  # Generate random color for each centroid
    centroid_colors.append(color)

# Create PointCloud with custom colors
custom_point_cloud = o3d.geometry.PointCloud()
custom_point_cloud.points = o3d.utility.Vector3dVector(downsampled_centroids_point_cloud.points)
custom_point_cloud.colors = o3d.utility.Vector3dVector(centroid_colors)

# Add PointClouds to visualization
vis.add_geometry(downsampled_point_cloud)
vis.add_geometry(custom_point_cloud)

# Add spheres around centroids
for centroid, color in zip(downsampled_centroids_point_cloud.points, centroid_colors):
    # Draw a sphere around each centroid
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.5)
    sphere.compute_vertex_normals()
    sphere.paint_uniform_color(color)
    sphere.translate(centroid)
    vis.add_geometry(sphere)
    
# # Generate 3D text point cloud for centroid coordinates
    label_text = f"({centroid[0]:.2f}, {centroid[1]:.2f}, {centroid[2]:.2f})"
    text_cloud = text_3d(label_text, centroid + np.array([0.5, 0.5, 0.5]))

    # Add text point cloud to visualization
    vis.add_geometry(text_cloud)

# Run the visualization
vis.run()

# centroid_colors = []
# for _ in range(len(downsampled_centroids_point_cloud.points)):
#     color = np.random.rand(3)  # Generate random color for each centroid
#     centroid_colors.append(color)

# # Create PointCloud with custom colors
# custom_point_cloud = o3d.geometry.PointCloud()
# custom_point_cloud.points = o3d.utility.Vector3dVector(downsampled_centroids_point_cloud.points)
# custom_point_cloud.colors = o3d.utility.Vector3dVector(centroid_colors)

# # Visualize point clouds
# vis.add_geometry(downsampled_point_cloud)
# vis.add_geometry(custom_point_cloud)
# vis.run()

# Visualize the total downsampled point cloud
# o3d.visualization.draw_geometries([downsampled_point_cloud, downsampled_centroids_point_cloud])

# # # Visualize the original point cloud and the centroids together
# o3d.visualization.draw_geometries([point_cloud, centroids_point_cloud])


NameError: name 'centroids' is not defined